# **Introduction to Data Science Final Project**

**Student Information:**

StudentID|Full Name
-|-
21127012|Tran Huy Ban
21127050|Tran Nguyen Huan 
21127143|Nguyen Minh Quan 
21127175|Le Anh Thu


## **Table of contents**

[Overview](#overview)

1. [Data Collection](#collect)
   

    b. [Collecting data](#collecting)
   
3. [Data Pre-processing](#process)

    a. [Pre-precessing](#preprocess)

    b. [Exploration](#exploration)
       
4. [Data Modeling](#modeling)

5. [Deploy Model](#deploy)

[References](#references)

## **Overview** <a name="overview"></a>

**Project Title:** Data Science Salary Predictor

<center>
<img src="https://www.nepalec.edu.np/storage/upload/23/06/16874373331583217311227.png" width="800">
</center>

**Overview:**
The Data Science Salary Predictor is a robust tool designed to empower data scientists during salary negotiations by providing accurate salary estimates based on key skills and job descriptions. This project involved scraping over 1000 data science job descriptions from [Glassdoor](#https://www.glassdoor.com/), extracting relevant features, optimizing machine learning models, and creating a user-friendly API for seamless access.

**Results:**
- The model achieved a high level of accuracy with a low MAE, providing data scientists with reliable salary estimates.
- The API allows users to input their specific skills and experience to receive instant salary predictions, aiding in negotiation strategies.

**Impact:**
- Empowers data scientists to negotiate competitive salaries based on industry standards and their unique skill set.
- Enhances transparency in the data science job market, allowing both job seekers and employers to make more informed decisions.

## **1. Data Collection** <a name="collect"></a>

## **2. Data Pre-processing** <a name="process"></a>

### **a. Pre-processing** <a name="preprocess"></a>

### **b. Exploration** <a name="exploration"></a>

## **3. Data Modeling** <a name="modeling"></a>

### **4. Deploy Model** <a name="deploy"></a>

## **References** <a name="references"></a>